In [99]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [100]:
f = pd.read_csv('data2.csv')


In [101]:
f= f.drop(['Unnamed: 12'], axis=1)

In [111]:
f.head(n=6)


,职业,ID,祝福,唱歌,一觉,三觉,其他,三攻,唱歌后三攻,力智总和,相对提升伤害,双C提升伤害(迫害奶萝),Unnamed: 13,常驻力智,常驻提升,常驻提升-2C,常驻提升倍率,爆发提升倍率,相对提升伤害-2C,力智总和-归一化,唱歌后三攻-归一化,常驻提升-归一化,相对提升伤害-归一化,常驻提升-2C-归一化,相对提升伤害2C-归一化
0,奶萝,凌采露华,35779,8944,41626,12071,777,4157,5196,99197,38422,35600,NaN,45500,20907.83208,18848.155224,9.534156,17.561747,35668.344918,0.918737,1.000000,0.919384,0.999766,0.920430,1.000000
1,奶萝,Deer °,44535,11133,40081,11623,599,3702,4627,107971,38431,35525,NaN,56267,22741.12082,20477.543199,10.370152,17.563170,35592.274462,1.000000,0.890493,1.000000,1.000000,1.000000,0.997867
2,奶妈,蝶舞花翩翩,43174,6476,39650,11498,991,3740,4301,101789,34983,34983,NaN,50641,20126.08262,20126.082620,9.177671,15.989195,35063.344540,0.942744,0.827752,0.885008,0.910281,0.982837,0.983038
3,奶萝,伊莉雅,43892,10973,38422,11142,599,3223,4028,105028,34586,32042,NaN,55464,20729.36712,18719.175133,9.452774,15.806542,32103.942460,0.972743,0.775212,0.911537,0.899951,0.914132,0.900068
4,奶妈,桃花舞翩翩,39212,5881,38029,11028,991,3470,3990,95141,31634,31634,NaN,46084,17994.63660,17994.636600,8.205713,14.460484,31710.973800,0.881172,0.767898,0.791282,0.823138,0.878750,0.889051
5,奶萝,惊星落鸿,38365,9591,37342,11202,742,3446,4307,97242,33683,31236,NaN,48698,19574.72230,17689.053784,8.926246,15.396300,31297.449094,0.900631,0.828907,0.860763,0.876454,0.863827,0.877457


In [103]:
carry_str = 4800
job_base = 950
carry_atk = 3000
carry_str_br = 2.31*(carry_str-job_base)+4581 + carry_str
carry_str_br

18274.5

In [104]:
def calNormalBuffStr(row):
    if row[u'祝福'] == '/':
        row[u'祝福'] =  0
    if row[u'唱歌'] == '/':
        row[u'唱歌'] =  0
    if row[u'其他'] == '/':
        row[u'其他'] = 0
    return int(row[u'祝福'])+int(row[u'唱歌'])+int(row[u'其他'])

def calNormalBuff(row, carry_number):
    if row[u'职业'] == '奶萝':
        atk = row[u'唱歌后三攻']*(1+(carry_number-1)*0.85)/carry_number
        str_int = row[u'常驻力智']*(1+(carry_number-1)*0.85)/carry_number
    else:
        atk = row[u'唱歌后三攻']
        str_int = row[u'常驻力智']
    return ((str_int+carry_str_br)/250) * (carry_atk+ atk)/100

def calBuffRate(row, carry_no):
    if (carry_no > 1 and row[u'职业'] == '奶萝'):
        atk = row[u'唱歌后三攻']*0.85
        str_int = row[u'常驻力智']*0.85
    else:
        atk = row[u'唱歌后三攻']
        str_int = row[u'常驻力智']
    carry_damage = carry_str_br / 250 * carry_atk / 100
    total_buff_damage = (carry_str_br+str_int)/250 * (carry_atk + atk) / 100
    return total_buff_damage / carry_damage

def calTotalBuffRate(row, carry_no):
    if (carry_no > 1 and row[u'职业'] == '奶萝'):
        atk = row[u'唱歌后三攻']*0.85
        str_int = row[u'常驻力智']*0.85+row[u'一觉']+row[u'三觉']
    else:
        atk = row[u'唱歌后三攻']
        str_int = row[u'力智总和']
    carry_damage = carry_str_br / 250 * carry_atk / 100
    total_buff_damage = (carry_str_br+str_int)/250 * (carry_atk + atk) / 100
    return total_buff_damage / carry_damage

def calTotallBuff(row, carry_number):
    atk_main = row[u'唱歌后三攻']
    str_main = row[u'力智总和']
    if row[u'职业'] == '奶萝':
        atk_oth = row[u'唱歌后三攻']*0.85
        str_oth = row[u'常驻力智']*0.85+row[u'一觉']+row[u'三觉']
    else:
        atk_oth = row[u'唱歌后三攻']
        str_oth = row[u'力智总和']
    main_buff = ((str_main+carry_str_br)/250) * (carry_atk+ atk_main)/100
    oth_buff = ((str_oth+carry_str_br)/250) * (carry_atk+ atk_oth)/100 * (carry_number-1)
    return (main_buff+oth_buff)/carry_number

def normalization(row, row_name, min_value, max_value):
    return (row[row_name]-min_value)/(max_value-min_value)

def normalization_zerobase(row,row_name, max_value):
    return (row[row_name])/(max_value)

In [105]:
f[u'常驻力智'] = f.apply(lambda x : calNormalBuffStr(x), axis=1)
f[u'常驻提升'] = f.apply(lambda x: calNormalBuff(x,1), axis=1)
f[u'常驻提升-2C'] = f.apply(lambda x: calNormalBuff(x,2), axis=1)
f[u'常驻提升倍率'] = f.apply(lambda x : calBuffRate(x, 1), axis=1)
f[u'爆发提升倍率'] = f.apply(lambda x : calTotalBuffRate(x, 1), axis=1)


f[u'相对提升伤害-2C'] = f.apply(lambda x: calTotallBuff(x,2), axis=1)

f[u'力智总和-归一化'] = f.apply(lambda x: normalization(x,u'力智总和', f[u'力智总和'].min(),f[u'力智总和'].max()), axis=1)

f[u'唱歌后三攻-归一化'] = f.apply(lambda x: normalization(x,u'唱歌后三攻',f[u'唱歌后三攻'].min(), f[u'唱歌后三攻'].max()), axis=1)

f[u'常驻提升-归一化'] = f.apply(lambda x: normalization(x,u'常驻提升',f[u'常驻提升'].min(), f[u'常驻提升'].max()), axis=1)

f[u'相对提升伤害-归一化'] = f.apply(lambda x: normalization(x,u'相对提升伤害',f[u'相对提升伤害'].min(), f[u'相对提升伤害'].max()), axis=1)
f[u'常驻提升-2C-归一化'] = f.apply(lambda x: normalization(x, u'常驻提升-2C', f[u'常驻提升-2C'].min(),f[u'常驻提升-2C'].max()), axis=1)
f[u'相对提升伤害2C-归一化'] = f.apply(lambda x: normalization(x,u'相对提升伤害-2C', f[u'相对提升伤害-2C'].min(), f[u'相对提升伤害-2C'].max()), axis=1)


In [106]:

f[u'力智总和-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'力智总和', f[u'力智总和'].max()), axis=1)

f[u'唱歌后三攻-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'唱歌后三攻', f[u'唱歌后三攻'].max()), axis=1)

f[u'常驻提升-归一化'] = f.apply(lambda x: normalization_zerobase(x, u'常驻提升', f[u'常驻提升'].max()), axis=1)

f[u'相对提升伤害-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'相对提升伤害', f[u'相对提升伤害'].max()), axis=1)
f[u'常驻提升-2C-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'常驻提升-2C', f[u'常驻提升-2C'].max()), axis=1)
f[u'相对提升伤害2C-归一化'] = f.apply(lambda x: normalization_zerobase(x,u'相对提升伤害-2C', f[u'相对提升伤害-2C'].max()), axis=1)

In [107]:
f[['ID',u'职业',u'力智总和-归一化', u'常驻提升-归一化',u'唱歌后三攻-归一化',u'相对提升伤害-归一化', u'相对提升伤害2C-归一化']]

,ID,职业,力智总和-归一化,常驻提升-归一化,唱歌后三攻-归一化,相对提升伤害-归一化,相对提升伤害2C-归一化
0,凌采露华,奶萝,0.918737,0.919384,1.000000,0.999766,1.000000
1,Deer °,奶萝,1.000000,1.000000,0.890493,1.000000,0.997867
2,蝶舞花翩翩,奶妈,0.942744,0.885008,0.827752,0.910281,0.983038
3,伊莉雅,奶萝,0.972743,0.911537,0.775212,0.899951,0.900068
4,桃花舞翩翩,奶妈,0.881172,0.791282,0.767898,0.823138,0.889051
5,惊星落鸿,奶萝,0.900631,0.860763,0.828907,0.876454,0.877457
6,RY丶god,奶爸,0.885377,0.761408,0.735758,0.806693,0.871284
7,羽生結弦,奶妈,0.885386,0.791519,0.732679,0.804819,0.869249
8,マネキン嬢,奶萝,0.922553,0.865858,0.776174,0.860920,0.861811
9,ジュリア,奶妈,0.789064,0.669214,0.852002,0.797715,0.861801


In [108]:
f.to_json('data_output.json',force_ascii=False,orient='records')

In [109]:
for r in rows.iterrows():
    data_tuple = r[1]
    result_text = """ {{
    value: [ {b} , {c},{d}, {e},{f}, {g} ] ,
    name: '{a}' }},
    """.format(a=data_tuple[0], b=data_tuple[2]*100, c=data_tuple[3]*100,
               d=data_tuple[4]*100, e=data_tuple[5]*100, f=data_tuple[6]*100,
              g=data_tuple[7]*100)
    print(  result_text)

 {
    value: [ 92.51154386669343 , 100.0,100.0, 100.0,100.0, 100.0 ] ,
    name: 'Deer °' },
    
 {
    value: [ 86.3481228668942 , 94.64606172185184,88.98687931477592, 98.8100656544247,91.56415222739884, 99.08346153719391 ] ,
    name: '蝶舞花翩翩' },
    
 {
    value: [ 100.0 , 86.68024212669809,86.77470817959086, 87.00290521899412,92.93566455530545, 93.22593125035426 ] ,
    name: '凌采露华' },
    
 {
    value: [ 80.86729572375026 , 97.65776823156388,91.6542838965611, 91.90277903709445,90.52504842171388, 90.72065970028909 ] ,
    name: '伊莉雅' },
    
 {
    value: [ 80.10439670748846 , 88.4645782774043,79.56275374951481, 88.34561883923709,82.7985133225148, 89.61019246851527 ] ,
    name: '桃花舞翩翩' },
    


In [110]:
for r in rows.iterrows():
    data_tuple = r[1]
    print("'"+data_tuple[0]+"',")

'Deer °',
'蝶舞花翩翩',
'凌采露华',
'伊莉雅',
'桃花舞翩翩',
